In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
badges = pd.read_json('/content/drive/My Drive/json data/badges.json')

In [ ]:
comments = pd.read_json('/content/drive/My Drive/json data/comments.json')

In [ ]:
posts = pd.read_json('/content/drive/My Drive/json data/posts.json')

In [ ]:
posts3 = pd.read_json('/content/drive/My Drive/json data/posts3.json')

In [ ]:
posts5 = pd.read_json('/content/drive/My Drive/json data/posts5.json')

In [ ]:
posts7 = pd.read_json('/content/drive/My Drive/json data/posts7.json')

In [ ]:
tags = pd.read_json('/content/drive/My Drive/json data/tags.json')

In [ ]:
users = pd.read_json('/content/drive/My Drive/json data/users.json')

In [ ]:
users5 = pd.read_json('/content/drive/My Drive/json data/users5.json')

In [ ]:
users9 = pd.read_json('/content/drive/My Drive/json data/users9.json')

In [ ]:
votes = pd.read_json('/content/drive/My Drive/json data/votes.json')

badges
créer une liste de liste de badges
pour chaque user, créer la liste de ses badges, et l'ajouter dans la grosse liste

In [ ]:
badgesList = []

In [ ]:
badges

,Id,UserId,Name,Date
0,1,5,Teacher,2010-07-19 19:39:07
1,2,6,Teacher,2010-07-19 19:39:07
2,3,8,Teacher,2010-07-19 19:39:07
3,4,23,Teacher,2010-07-19 19:39:07
4,5,36,Teacher,2010-07-19 19:39:07
...,...,...,...,...
79846,92236,55744,Student,2014-09-13 23:25:21
79847,92237,1118,Nice Answer,2014-09-14 00:09:35
79848,92238,1118,Enlightened,2014-09-14 01:18:29
79849,92239,55746,Student,2014-09-14 01:41:18


In [ ]:
# retourne la liste de tous les badges d'un utilisateur donné, et supprime le le UserId (donnée non nécessaire, une fois la table fusionnée dans User)
def getBadges(userId):
  liste = badges[badges.UserId == userId]
  #supprimer la colonne userId
  del liste['UserId']
  return liste


In [ ]:
def badges_users_fusion():
  Liste = []
  n = len(users)
  i=0
  for i in range(0,n):
    uId = int(users['Id'][i:i+1])
    userBadges = getBadges(uId)
    Liste.append(userBadges)
  return Liste

In [ ]:
badgesList = badges_users_fusion()
users['Badges'] = badgesList
posts.to_json(r'/content/drive/My Drive/json data/posts2.json', orient='records')

In [ ]:
#malheureusement la méthode to_json écrit tout sur une seule et même ligne, ce qui complique la lecture afin de vérifier que les modifications effectuées sont bonnes. il est donc nécessaire de rajouter des sauts à la ligne pour que le dataset soit lisible
def reformatage(inputFile, outputFile):
  fin = open(inputFile, "rt")
  fout = open(outputFile, "wt")

  for line in fin:
  	fout.write(line.replace(']},{"Id":', ']},\n{"Id":'))
  fin.close()
  fout.close()

Tags

In [ ]:
def getTags(postId):
  liste = tags[tags.ExcerptPostId == postId]
  del liste['ExcerptPostId']
  return liste

In [ ]:
def Tags_posts_fusion():
  Liste = []
  n = len(posts)
  i=0
  for i in range(0,n):
    pId = int(posts['Id'][i:i+1])
    userTags = getTags(pId)
    Liste.append(userTags)
  return Liste

In [ ]:
tagsList = Tags_posts_fusion()
posts['Tags'] = tagsList
posts

votes


In [ ]:
def getVotes(postId):
  liste = votes[votes.PostId == postId]
  del liste['PostId']
  return liste


In [ ]:
def Votes_posts_fusion():
  Liste = []
  n = len(posts)
  i=0
  for i in range(0,n):
    pId = int(posts['Id'][i:i+1])
    userVotes = getVotes(pId)
    Liste.append(userVotes)
  return Liste

In [ ]:
votesList = Votes_posts_fusion()
posts['Votes'] = votesList
posts["Badges"]
posts.to_json(r'/content/drive/My Drive/json data/votes2.json', orient='records')

modification du format des tags
dans le dataset, les tags sont ordonnés de la manière suivante : "Tag":"<tag1><tag2><tag3>..." ce qui n'est pas évident à exploiter
le but des méthodes ci dessous est de les ordonner avec le format suivant : "Tag": ["tag1", "tag2","tag3"...]

In [ ]:
def cleanTags(string):
  liste = string.split(">")
  if (len(liste) > 1):
    n = len(liste)
    del liste[n-1]
    while liste[0][0] != "<":
      liste[0] = liste[0][1:]
    liste[0] = liste[0][1:]
    for i in range(1,n-1):
      liste[i] = liste[i][1:]
  else:
    liste = []

  return liste

In [ ]:
def correction_tags():
  Liste = []
  n= len(posts3)
  for i in range(0, n):
    string = str(posts3["Tags"][i:i+1])
    tagList = cleanTags(string)
    Liste.append(tagList)
  del posts3["Tags"]
  posts3['Tags'] = Liste

In [ ]:
correction_tags()
posts3.to_json(r'/content/drive/My Drive/json data/post4.json', orient='records')

Ajout des postId dans users

In [ ]:
def addPostIds():
  n = len(users)
  Liste = []
  for i in range(0,n):
    uid = int(users['Id'][i:i+1])
    pids = posts3["Id"][posts3.OwnerUserId == uid]
    Liste.append(pids)
  users['PostIds'] = Liste

In [ ]:
addPostIds()
users.to_json(r'/content/drive/My Drive/json data/users4.json', orient='records')

table comments spéciale

In [ ]:
commentUsers = comments[["Id","UserId"]]

In [ ]:
commentUsers = commentUsers.rename(columns={"Id": "CommentId", "UserId": "UserId"})
commentUsers

ajout des commentIds dans user

In [ ]:
def addCommentIds():
  Liste = []
  n = len(users5)
  for i in range(0,n):
    uid = int(users5["Id"][i:i+1])
    cids = commentUsers["CommentId"][commentUsers.UserId == uid]
    Liste.append(cids)
  users5['CommentId'] = Liste


In [ ]:
addCommentIds()
users5.to_json(r'/content/drive/My Drive/json data/users6.json', orient='records')

dernière étape : mettre les comments dans posts

In [ ]:
def addComments():
  Liste = []
  n = len(posts5)
  for i in range(0,n):
    pid = int(posts5["Id"][i:i+1])
    commentsList = comments[comments.PostId == pid]
    Liste.append(commentsList)
  posts5['Comments'] = Liste

In [ ]:
addComments()
posts5.to_json(r'/content/drive/My Drive/json data/posts6.json', orient='records')

remettre les user id dans les comments

In [ ]:
def addPostIdsInUserComments():
  n = len(users7)
  Liste = []
  for i in range(0,n):
    uid = int(users7["Id"][i:i+1])
    commentsList = comments[comments.UserId == uid]
    commentIds = commentsList[["Id","PostId"]]
    Liste.append(commentIds)
  users7['CommentId'] = Liste

In [ ]:
commentIds = commentsList[["Id","PostId"]]

In [ ]:
addPostIdsInUserComments()

In [ ]:
users7.to_json(r'/content/drive/My Drive/json data/users8.json', orient='records')